## setup

In [1]:
import Interface as I

/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:13: UserWarning: environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. Falling back to default configuration.
  warnings.warn('environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. ' +
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:33: UserWarning: Using file based locking.Please be careful on nfs mounts as file based locking has issues in this case.
  warnings.warn('Using file based locking.'


trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'spock', 'port': 8885, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'localhost', 'port': 6379, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'file'}
Current version: heads/vcbm-2023+0.ge0cff175.dirty
Current pid: 313687


/scratch/visualn/bzfharth/in-silico-install-dir/anaconda_isf3/lib/python3.8/site-packages/pandas_msgpack-0.1.4+14.gfcb0471-py3.8-linux-x86_64.egg/pandas_msgpack/packers.py:78: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Timestamp, Period, Series, DataFrame,  # noqa
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/visualize/cell_morphology_visualizer.py:29: ImportWarning: Dash could not be imported. Interactive visualisations will not work.
  warnings.warn(


Loading mechanisms:
setting up local multiprocessing framework ... done


Loaded modules with __version__ attribute are:
IPython: 7.19.0, Interface: heads/vcbm-2023+0.ge0cff175.dirty, PIL: 8.2.0, _csv: 1.0, _ctypes: 1.1.0, _curses: b'2.2', _decimal: 1.70, argparse: 1.1, backcall: 0.2.0, blosc: 1.10.2, bluepyopt: 1.9.126, bottleneck: 1.3.2, cffi: 1.14.3, click: 7.1.2, cloudpickle: 1.6.0, colorama: 0.4.4, csv: 1.0, ctypes: 1.1.0, cycler: 0.10.0, cytoolz: 0.11.0, dask: 2.30.0, dateutil: 2.8.1, deap: 1.3, decimal: 1.70, decorator: 4.4.2, distributed: 2.30.1, distutils: 3.8.5, fsspec: 0.8.3, future: 0.18.2, ipykernel: 5.3.4, ipython_genutils: 0.2.0, ipywidgets: 7.5.1, jedi: 0.17.1, jinja2: 2.11.2, joblib: 0.17.0, json: 2.0.9, jupyter_client: 6.1.7, jupyter_core: 4.6.3, kiwisolver: 1.3.0, logging: 0.5.1.2, markupsafe: 1.1.1, matplotlib: 3.3.2, mkl: 2.3.0, neuron: 7.8.2, numexpr: 2.7.1, numpy: 1.19.2, pandas: 1.1.3, pandas_msgpack: 0.1.4+14.gfcb0471.dirty, parameters: 0.2.1, parso: 0.7.0, pas

/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/Interface.py:190: UserWarning:

The source folder has uncommited changes!



In [2]:
postfix = "_2023-06-22"
mdb = I.ModelDataBase('/scratch/visualn/bzfharth/simulation_data'+postfix)
print(mdb.keys())

['89', '90', 'WR69', '88', '93', '84', '87', 'WR64', '85', '86', '91', 'WR71', 'examplary_models', 'df_all', 'run_gillis_protocol_v1', 'all_models_py3', 'run_gillis_protocol_v1__all_models_genetic_py3', 'all_models_genetic_py3', 'all_models_genetic_py2_1', 'all_models_genetic_py2_2', 'all_models_RW_BAC2step_seed_11_filtered', 'all_models_RW_BAC2step_seed_17_filtered', 'all_models_RW_BAC2step_seed_1_filtered']


In [3]:
import os
import shutil

def makeCleanDir(dirname):
    if(os.path.exists(dirname)):
        shutil.rmtree(dirname, ignore_errors=False, onerror=None)
    os.mkdir(dirname)

selected_dataset = "examplary_models"
csv_outfolder = os.path.join('/scratch/visualn/bzfharth/simulation_data_postprocessed', selected_dataset+postfix)

makeCleanDir(csv_outfolder)

In [4]:
# init
import pickle
import biophysics_fitting.hay_evaluation
import project_specific_ipynb_code.biophysical_models
from project_specific_ipynb_code.biophysical_models import get_model_ddf_from_encapsulating_mdb, objectives_BAC
from project_specific_ipynb_code.biophysical_models import get_ddf_selected
from project_specific_ipynb_code.hot_zone import get_cell_object_from_hoc
from project_specific_ipynb_code.biophysical_models import min_max_plot, normalize_params
from project_specific_ipynb_code.biophysical_models import CurrentAnalysis
from project_specific_ipynb_code.biophysical_models import modify_simulator_to_record_apical_dendrite_conductances

In [5]:
# init
objectives_BAC = project_specific_ipynb_code.biophysical_models.objectives_BAC
objectives_step = project_specific_ipynb_code.biophysical_models.objectives_step
objectives_2BAC = ['1BAC_APheight', '1BAC_ISI', '1BAC_ahpdepth', '1BAC_caSpike_height', '1BAC_caSpike_width', '1BAC_spikecount', '2BAC_APheight', '2BAC_ISI', '2BAC_ahpdepth', '2BAC_caSpike_height', '2BAC_caSpike_width', '2BAC_spikecount', 'bAP_APheight', 'bAP_APwidth', 'bAP_att2', 'bAP_att3', 'bAP_spikecount']

In [6]:
# they are the same for all optimizations with the exception of CaDynamics_E2 (py2 simulations) 
# and CaDynamics_E2_v2 (py3 simulations)

params_py2 = ['ephys.CaDynamics_E2.apic.decay',
       'ephys.CaDynamics_E2.apic.gamma', 'ephys.CaDynamics_E2.axon.decay',
       'ephys.CaDynamics_E2.axon.gamma', 'ephys.CaDynamics_E2.soma.decay',
       'ephys.CaDynamics_E2.soma.gamma', 'ephys.Ca_HVA.apic.gCa_HVAbar',
       'ephys.Ca_HVA.axon.gCa_HVAbar', 'ephys.Ca_HVA.soma.gCa_HVAbar',
       'ephys.Ca_LVAst.apic.gCa_LVAstbar', 'ephys.Ca_LVAst.axon.gCa_LVAstbar',
       'ephys.Ca_LVAst.soma.gCa_LVAstbar', 'ephys.Im.apic.gImbar',
       'ephys.K_Pst.axon.gK_Pstbar', 'ephys.K_Pst.soma.gK_Pstbar',
       'ephys.K_Tst.axon.gK_Tstbar', 'ephys.K_Tst.soma.gK_Tstbar',
       'ephys.NaTa_t.apic.gNaTa_tbar', 'ephys.NaTa_t.axon.gNaTa_tbar',
       'ephys.NaTa_t.soma.gNaTa_tbar', 'ephys.Nap_Et2.axon.gNap_Et2bar',
       'ephys.Nap_Et2.soma.gNap_Et2bar', 'ephys.SK_E2.apic.gSK_E2bar',
       'ephys.SK_E2.axon.gSK_E2bar', 'ephys.SK_E2.soma.gSK_E2bar',
       'ephys.SKv3_1.apic.gSKv3_1bar', 'ephys.SKv3_1.apic.offset',
       'ephys.SKv3_1.apic.slope', 'ephys.SKv3_1.axon.gSKv3_1bar',
       'ephys.SKv3_1.soma.gSKv3_1bar', 'ephys.none.apic.g_pas',
       'ephys.none.axon.g_pas', 'ephys.none.dend.g_pas',
       'ephys.none.soma.g_pas', 'scale_apical.scale']

params_py3 = [p.replace('ephys.CaDynamics_E2', 'ephys.CaDynamics_E2_v2') for p in params_py2]

In [7]:
df = mdb[selected_dataset]
print(df.shape)

(32, 116)


In [8]:
feature_names = set()
blocked = set(['scale_apical.scale', 'model_id', 'type_',
       'morphology', 'lv', 'seed', 'max_', 'gen',
       'hot_zone.outsidescale_sections', 'BAC.hay_measure.recSite',
       'BAC.stim.dist', 'bAP.hay_measure.recSite1', 'bAP.hay_measure.recSite2',
       'hot_zone.min_', 'hot_zone.max_', 'morphology.filename', 'sort_column'])
for col_name in df.columns:
    if("ephys" not in col_name):
        feature_names.add(col_name)

feature_names = sorted(feature_names-blocked)
feature_names

['1BAC_APheight',
 '1BAC_ISI',
 '1BAC_ahpdepth',
 '1BAC_caSpike_height',
 '1BAC_caSpike_width',
 '1BAC_spikecount',
 '2BAC_APheight',
 '2BAC_ISI',
 '2BAC_ahpdepth',
 '2BAC_caSpike_height',
 '2BAC_caSpike_width',
 '2BAC_spikecount',
 'AI1',
 'AI2',
 'AI3',
 'APh1',
 'APh2',
 'APh3',
 'APw1',
 'APw2',
 'APw3',
 'BAC_APheight',
 'BAC_ISI',
 'BAC_ahpdepth',
 'BAC_caSpike_height',
 'BAC_caSpike_width',
 'BAC_spikecount',
 'DI1',
 'DI2',
 'DI3',
 'ISIcv1',
 'ISIcv2',
 'ISIcv3',
 'Simulator',
 'TTFS1',
 'TTFS2',
 'TTFS3',
 'bAP_APheight',
 'bAP_APwidth',
 'bAP_att2',
 'bAP_att3',
 'bAP_spikecount',
 'description',
 'fAHPd1',
 'fAHPd2',
 'fAHPd3',
 'iloc',
 'mf1',
 'mf2',
 'mf3',
 'morphology_id',
 'optim_id',
 'sAHPd1',
 'sAHPd2',
 'sAHPd3',
 'sAHPt1',
 'sAHPt2',
 'sAHPt3',
 'value']

In [9]:
parameters = df[params_py3]
parameters.to_csv(os.path.join(csv_outfolder, "parameters.csv"))
parameters

,ephys.CaDynamics_E2_v2.apic.decay,ephys.CaDynamics_E2_v2.apic.gamma,ephys.CaDynamics_E2_v2.axon.decay,ephys.CaDynamics_E2_v2.axon.gamma,ephys.CaDynamics_E2_v2.soma.decay,ephys.CaDynamics_E2_v2.soma.gamma,ephys.Ca_HVA.apic.gCa_HVAbar,ephys.Ca_HVA.axon.gCa_HVAbar,ephys.Ca_HVA.soma.gCa_HVAbar,ephys.Ca_LVAst.apic.gCa_LVAstbar,...,ephys.SKv3_1.apic.gSKv3_1bar,ephys.SKv3_1.apic.offset,ephys.SKv3_1.apic.slope,ephys.SKv3_1.axon.gSKv3_1bar,ephys.SKv3_1.soma.gSKv3_1bar,ephys.none.apic.g_pas,ephys.none.axon.g_pas,ephys.none.dend.g_pas,ephys.none.soma.g_pas,scale_apical.scale
0,55.131259,0.000536,143.024062,0.004450,668.441526,0.005497,0.004898,0.000585,0.000026,0.000359,...,0.012039,0.013580,-2.546384,1.237247,1.036869,0.000038,0.000024,0.000094,0.000021,2.994014
1,101.176488,0.001968,57.969940,0.008433,512.051030,0.000931,0.000009,0.000305,0.000209,0.196167,...,0.000001,0.312149,-2.812210,0.841482,0.972746,0.000062,0.000039,0.000048,0.000029,2.987206
2,101.332202,0.007168,136.483383,0.008268,498.825160,0.001094,0.000177,0.000242,0.000142,0.130357,...,0.000429,0.090515,-2.399174,0.926303,0.972630,0.000054,0.000039,0.000045,0.000027,2.980477
3,87.295158,0.004341,757.944338,0.013519,135.919490,0.005105,0.000941,0.000628,0.000274,0.121764,...,0.000085,0.000153,-2.906186,1.069495,0.761588,0.000071,0.000047,0.000036,0.000036,2.998521
4,48.574214,0.029706,190.367140,0.002876,951.036059,0.001171,0.002975,0.000465,0.000043,0.191190,...,0.000004,0.017647,-2.113520,1.253560,0.901975,0.000071,0.000026,0.000043,0.000032,2.990184
5,85.506579,0.007592,136.408178,0.008433,537.902433,0.000540,0.000175,0.000248,0.000251,0.155927,...,0.004396,0.125731,-2.701750,1.046983,0.972381,0.000065,0.000049,0.000053,0.000027,2.980641
6,89.777263,0.004316,959.500012,0.028393,185.365328,0.004176,0.000954,0.000840,0.000276,0.115878,...,0.009458,0.000335,-2.776904,1.470798,0.572653,0.000071,0.000048,0.000037,0.000037,3.000000
7,87.182156,0.004337,774.938566,0.016378,169.444610,0.005045,0.000926,0.000762,0.000269,0.121763,...,0.000298,0.000121,-2.993878,1.443312,0.752095,0.000066,0.000037,0.000038,0.000036,2.677242
8,67.918140,0.011929,550.333289,0.009246,211.024903,0.001198,0.000527,0.000041,0.000700,0.184083,...,0.000036,0.003385,-1.681924,1.579979,0.916289,0.000085,0.000032,0.000032,0.000026,2.999581
9,69.740044,0.000561,383.270167,0.001544,101.112087,0.002931,0.003373,0.000516,0.000191,0.056023,...,0.000023,0.838735,-1.766424,0.495785,0.901977,0.000051,0.000045,0.000094,0.000027,2.938875


In [10]:
features = df[feature_names]
features.to_csv(os.path.join(csv_outfolder, "features.csv"))
features

,1BAC_APheight,1BAC_ISI,1BAC_ahpdepth,1BAC_caSpike_height,1BAC_caSpike_width,1BAC_spikecount,2BAC_APheight,2BAC_ISI,2BAC_ahpdepth,2BAC_caSpike_height,...,mf3,morphology_id,optim_id,sAHPd1,sAHPd2,sAHPd3,sAHPt1,sAHPt2,sAHPt3,value
0,0.711611,2.936480,0.334893,2.999421,2.594488,0.0,0.971383,1.380768,0.412330,2.017817,...,0.225002,WR64,step,1.176865,0.604594,0.379574,0.204396,0.648574,2.786473,0.991423
1,1.225208,1.305037,0.258493,2.986296,0.929134,0.0,1.330839,2.581895,0.279313,2.396132,...,3.375034,WR64,step,0.765580,0.154005,0.148103,1.269760,1.667475,3.574751,0.961259
2,0.926502,0.191969,0.177453,2.018352,0.862205,0.0,1.093507,2.816720,0.204386,2.298054,...,1.575016,WR64,step,0.807851,0.212916,0.082244,2.898941,2.758399,4.101134,0.052416
3,0.909364,1.762358,0.391443,0.836715,1.078740,0.0,1.237797,0.982153,0.371606,0.100796,...,1.125011,WR64,step,0.958147,0.426572,0.477659,1.927264,1.587542,3.302698,0.999981
4,0.823408,2.261360,0.284071,0.952462,2.791339,0.0,1.136115,0.042855,0.278963,0.291583,...,1.125011,WR64,step,0.896657,0.338380,0.206439,0.522139,0.612152,2.641232,0.999472
5,0.915606,0.558882,0.282351,1.347697,0.389764,0.0,1.151916,1.745333,0.299415,1.940510,...,2.025020,WR64,step,0.749517,0.132900,0.158104,1.635876,2.692857,3.991437,0.980978
6,1.124353,0.720324,0.386223,0.504938,1.216535,0.0,1.311015,1.114242,0.420929,0.614925,...,4.050041,WR64,step,0.517992,0.139968,0.273823,0.862398,1.515394,1.806465,3.000000
7,1.122521,1.410121,0.258115,1.768310,2.515748,0.0,1.488600,1.613244,0.283686,1.337178,...,1.575016,WR64,step,0.885611,0.355792,0.338798,1.926031,1.901131,2.601369,2.677242
8,0.890802,0.749677,0.100356,3.054818,2.259843,0.0,1.136658,2.669954,0.087104,2.668909,...,2.700027,WR64,step,0.853335,0.287733,0.277460,0.773187,3.321700,2.217992,3.673609
9,1.049423,1.835740,0.399196,2.723234,0.043307,0.0,1.461706,2.347071,0.453354,0.837882,...,2.700027,WR64,step,0.976154,0.307192,0.309385,1.867153,1.774414,2.734112,2.734112
